In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("Data/kitCapstoneJan24_data.csv", sep=";")

## selecting the country that we want

In [ ]:
selected_country = 'Country A'
#Get rows with for country A

df_country_a = df[df['countryname'] == selected_country]
df_country_a

## building a additional feature for months (1-12)

important for meanbymonth

In [ ]:
df_country_a['month_normalized'] = df_country_a['month'] % 12
df_country_a

df_country_a = df_country_a[['month_normalized','month', 'countryname'] + [col for col in df_country_a.columns if col not in ['month_normalized','month', 'countryname']]]

df_country_a

## select an vehiclegroup

In [ ]:
selcted_vehiclegroup = 'vehiclegroup01'

# delete all columns that are for a diffe

df1 = df_country_a

# create new dataframe with month and all columns containing vehiclegroup01
df2 = df1.filter(regex=selcted_vehiclegroup)
df2.head()

# drop columns in df1 which contain vehiclegroup
df3 = df1.drop(df1.filter(regex='vehiclegroup').columns, axis=1)
df3.head()

# merge df2 and df3 by index
df4 = df2.merge(df3, left_index=True, right_index=True)
df4.head()

# is there a column month in df4?
df4.columns.str.contains('month').any()
# column month should be the first column, second column is countryname
df4 = df4[['month_normalized','month', 'countryname'] + [col for col in df4.columns if col not in ['month_normalized','month', 'countryname']]]

In [ ]:
# drop rows after row 258
test = df4.drop(df4.index[258:])

## build different dataframes for internal and external data

In [ ]:
# dataframe with first 18 columns (internal data)
df_internal = test.iloc[:, :21]
df_internal.head()

In [ ]:
# dataframe with external features
df_external = test.iloc[:, 21:]
df_external.head()

# building target values


In [1]:
#take every row from 50 to 255 and 
#X = df_internal.iloc[49:255,:-3]
X = df_internal.iloc[49:255, ~df_internal.columns.str.contains('sales_actuals_monthly__vehiclegroup01__orderintake')]
y = df_internal.iloc[49:255, 3]

X_with_target = df_internal.iloc[49:255,:]


NameError: name 'df_internal' is not defined

## testing wheel

In [2]:
checkup_features = [['quota__vehiclegroup01__aq'],['quota__vehiclegroup01__aq']]

mean_per_month = True
interpolation_linear = True
fill_zero = True

for feature in checkup_features:
    if mean_per_month:
        # add mean per month
        # Fill in missing values using the corresponding month values
        month_means = {}
        for i in range(len(X)):
            month = X.iloc[i]['month_normalized']
            month_values = X.loc[X['month_normalized'] == month, feature[0]]
            if not month_values.isnull().values.all():
                month_means[month] = month_values.mean()

        # Fill in missing values using the corresponding month values
        X[feature[0]+'_meanbymonth']=X[feature[0]].loc[:].fillna(X['month_normalized'].map(month_means))

    if interpolation_linear:
        X[feature[0]+'_interpolated_linear']=X[feature[0]].interpolate(method='linear', limit_direction = 'both')

    if fill_zero:
       X[feature[0]+'_fill_zero']=X[feature[0]].fillna(0)

NameError: name 'X' is not defined

## SHAP

In [ ]:
model = xgboost.XGBRegressor().fit(X, y)

print('Accuracy Score of the Model is: ',model.score(X, y))

explainer = shap.Explainer(model)
shap_values = explainer(X)